## Packages

In [6]:
## Pandas, Numpy, time, warnings

import pandas as pd
import numpy as np
import time
import warnings

pd.set_option('display.max_colwidth', None)
warnings.filterwarnings('ignore')

## Inputs

> Team Standings and upcoming matches as on 04/12/2022 - 9 AM IST

In [7]:
## Current points table, Upcoming matches schedule and no of matches to consider for analysis

Team_Points_table_df = pd.read_csv('Current_Points_Table.csv')
Week1_Matches_table_df = pd.read_csv('UpcomingFixturesTable.csv')
No_of_matches_to_be_considered = 6

## Code for calculations

> Total number of possible combinations of results for the next 16 matches - 190 Bilion (190,734,863,280)

> For simplicity, I have considered only the next 6 matches - which helps the teams to not look too forward and focus on the upcoming games

In [8]:
## Compute all combinations of results based on the fixtures
temp_df = pd.DataFrame(columns=['Match_No', 'Winning_Team', 'Draw', 'Linked_Prev_Match_Index', 'Close', 'Losing_Team'])
new_combo_considered = -1

start_time = time.time()
for curr_match in Week1_Matches_table_df[Week1_Matches_table_df['Match_No'] == 1].values:
    temp_df.loc[len(temp_df.index)] = [curr_match[2], curr_match[0], 0, new_combo_considered, 0, curr_match[1]]
    temp_df.loc[len(temp_df.index)] = [curr_match[2], curr_match[1], 0, new_combo_considered, 0, curr_match[0]]
    temp_df.loc[len(temp_df.index)] = [curr_match[2], curr_match[0], 0, new_combo_considered, 1, curr_match[1]]
    temp_df.loc[len(temp_df.index)] = [curr_match[2], curr_match[1], 0, new_combo_considered, 1, curr_match[0]]  
    temp_df.loc[len(temp_df.index)] = [curr_match[2], str(curr_match[0]) + "_" + str(curr_match[1]), 1, new_combo_considered, 0, str(curr_match[0]) + "_" + str(curr_match[1])]

    temp_combo_to_be_considered = [new_combo_considered]

prev_combo_to_be_considered = temp_combo_to_be_considered

end_time = time.time()
time_diff = end_time - start_time
print("Match 1", "  -----  ",  np.round(time_diff, 2), 's')

for curr_match_no in range(2, No_of_matches_to_be_considered+1):
    for curr_match in Week1_Matches_table_df[Week1_Matches_table_df['Match_No'] == curr_match_no].values:
        temp_combo_to_be_considered = []
        for prev_combo in prev_combo_to_be_considered:
            for curr_combo in temp_df[temp_df['Linked_Prev_Match_Index'] == prev_combo].reset_index().values:
                temp_df.loc[temp_df.shape[0]] = [curr_match[2], curr_match[0], 0, curr_combo[0], 0, curr_match[1]]
                temp_df.loc[temp_df.shape[0]] = [curr_match[2], curr_match[1], 0, curr_combo[0], 0, curr_match[0]]
                temp_df.loc[len(temp_df.index)] = [curr_match[2], curr_match[0], 0, curr_combo[0], 1, curr_match[1]]
                temp_df.loc[len(temp_df.index)] = [curr_match[2], curr_match[1], 0, curr_combo[0], 1, curr_match[0]]  
                temp_df.loc[temp_df.shape[0]] = [curr_match[2], str(curr_match[0]) + "_" + str(curr_match[1]), 1, curr_combo[0], \
                    0, str(curr_match[0]) + "_" + str(curr_match[1])]

                temp_combo_to_be_considered.append(curr_combo[0])

    prev_combo_to_be_considered = temp_combo_to_be_considered

    end_time = time.time()
    time_diff = end_time - start_time
    print("Match ", curr_match_no, "  -----  ",  np.round(time_diff, 2), 's')

print('Total number of possible combinations - ', str(temp_df.shape[0]) )

Match 1   -----   0.03 s
Match  2   -----   0.18 s
Match  3   -----   0.83 s
Match  4   -----   4.17 s
Match  5   -----   21.58 s
Match  6   -----   109.33 s
Total number of possible combinations -  19530


In [9]:
## Evaluate points for all combinations

Team_Points_table_df_copy = Team_Points_table_df[['Team_Name', 'Short_Name', 'Points']].copy()
Team_Points_table_df_copy.rename(columns={'Points': -1}, inplace = True)

for curr_match_combo in temp_df.reset_index().values:

    if curr_match_combo[3] == 1:
        draw_teams = curr_match_combo[2].split('_')
    else:
        draw_teams = []
    if curr_match_combo[3] == 0:
        win_team = curr_match_combo[2]
        lose_team = curr_match_combo[6]
    else:
        win_team = ''
        lose_team = ''
    curr_match_combo_points = []
    for curr_team_point in Team_Points_table_df_copy[['Short_Name',curr_match_combo[4]]].values:
#        if curr_team_point[0] == curr_match_combo[2]:
        if curr_team_point[0] in draw_teams:
            curr_match_combo_points.append(curr_team_point[1] + 3)
        elif curr_team_point[0] == win_team:
            curr_match_combo_points.append(curr_team_point[1] + 5)
        elif curr_team_point[0] == lose_team:
            if curr_match_combo[5] == 1:
                curr_match_combo_points.append(curr_team_point[1] + 1)
            else:
                curr_match_combo_points.append(curr_team_point[1])
        else:
            curr_match_combo_points.append(curr_team_point[1])

    Team_Points_table_df_copy.loc[:, curr_match_combo[0]] = curr_match_combo_points


In [10]:
## Evaluate Table position for each combination

Team_Points_table_df_copy_rank = Team_Points_table_df_copy[['Team_Name', 'Short_Name']].copy()
for curr_match_combo_no in Team_Points_table_df_copy.drop(columns = ['Team_Name', 'Short_Name']).columns:
    if curr_match_combo_no > max(prev_combo_to_be_considered):
        Team_Points_table_df_copy_rank.loc[:, curr_match_combo_no] = Team_Points_table_df_copy[curr_match_combo_no].rank(method='max', ascending=False).values

## Key Functions

In [11]:
## A crosstab with the chances for all teams

def compute_chances_for_all_teams():

    chances_df = pd.DataFrame(index=range(1,Team_Points_table_df.shape[0]+1), columns=Team_Points_table_df_copy_rank['Short_Name'].values)
    chances_df.fillna('-', inplace=True)

    for curr_team in Team_Points_table_df_copy_rank['Short_Name'].values:
        curr_team_df = Team_Points_table_df_copy_rank[Team_Points_table_df_copy_rank['Short_Name'] == curr_team].drop(columns=['Team_Name', 'Short_Name']).T
        curr_team_value_counts_df = curr_team_df[curr_team_df.columns[0]].value_counts(normalize=True).reset_index().values
        for curr_team_value_count in curr_team_value_counts_df:
            chances_df.loc[curr_team_value_count[0], curr_team] = str(int(curr_team_value_count[1]*100)) + '%'

    return chances_df

In [12]:
## A simple statistic about the team of interest

def find_range_points(curr_team_short_name):
    curr_index_no = Team_Points_table_df_copy_rank[Team_Points_table_df_copy_rank['Short_Name'] == curr_team_short_name].index[0]
    curr_team_name = Team_Points_table_df_copy_rank.iloc[curr_index_no]['Team_Name']
    curr_df = Team_Points_table_df_copy_rank.drop(columns=['Team_Name', 'Short_Name']).iloc[curr_index_no]
    top_rank, avg_rank, mid_rank, worst_rank = curr_df.min(), curr_df.mean(), curr_df.median(), curr_df.max()

    team_pos_dict = {'Parameter':['Team_Name', 'Top_Position', 'Worst_Position', 'Avg_Position', 'Mid_Position'],
    'Value': [curr_team_name, top_rank, worst_rank, avg_rank, mid_rank]}
    team_pos_df = pd.DataFrame(data = team_pos_dict,columns=['Parameter', 'Value'])

    return team_pos_df

In [13]:
## Get Match wise favorable results to get the expected result

def define_curr_combo_result(curr_combo_result):
    curr_match_df = Week1_Matches_table_df[Week1_Matches_table_df['Match_No'] == curr_combo_result['Match_No']]
    Team_1, Team_2 = curr_match_df['Team_1'].iloc[0], curr_match_df['Team_2'].iloc[0]
    
    if curr_combo_result['Draw'] == 1:
        return f"Draw match between {Team_1} and {Team_2}"
    else:
        if curr_combo_result['Close'] == 1:
            return f"Close match with {curr_combo_result['Winning_Team']} winning against {curr_combo_result['Losing_Team']}"
        else:
            return f"Easy win for {curr_combo_result['Winning_Team']} against {curr_combo_result['Losing_Team']}"

def left_align(df):
    try:
        left_aligned_df = df.style.set_properties(**{'text-align': 'left'})
        left_aligned_df = left_aligned_df.set_table_styles(
            [dict(selector='th', props=[('text-align', 'left')])]
        )
    except:
        left_aligned_df = df
    return left_aligned_df

def Define_Favorable_Scenario(Exp_Team, Exp_Ranking):
    Exp_Team_Rankings = Team_Points_table_df_copy_rank[Team_Points_table_df_copy_rank['Short_Name'] == Exp_Team].drop(columns=['Team_Name', 'Short_Name']).T
    Combo_ExpTeam_Ranking = list(Exp_Team_Rankings[Exp_Team_Rankings[Exp_Team_Rankings.columns[0]] == Exp_Ranking].index)
    changing_Total_Matches = No_of_matches_to_be_considered

    if len(Combo_ExpTeam_Ranking) == 0:
        finding_root_path_Exp_Team_df = finding_root_path_Exp_Team_df = pd.DataFrame(Combo_ExpTeam_Ranking, columns = {f'Match_{changing_Total_Matches}'})
        Matchwise_Expected_Result_df = f"No chance for {Team_Points_table_df[Team_Points_table_df['Short_Name'] == Exp_Team]['Team_Name'].iloc[0]} Team to be at Position No - {Exp_Ranking} by the end of {No_of_matches_to_be_considered} matches"

    else:
        Matchwise_Expected_Result_dict = dict()
        versus_strings = []
        while changing_Total_Matches > 0:
            curr_match_df = Week1_Matches_table_df[Week1_Matches_table_df['Match_No'] == changing_Total_Matches]
            versus_string = str(curr_match_df['Team_1'].iloc[0]) + " vs " + str(curr_match_df['Team_2'].iloc[0])
            versus_strings.append(versus_string)

            if changing_Total_Matches == No_of_matches_to_be_considered:
                finding_root_path_Exp_Team_df = pd.DataFrame(Combo_ExpTeam_Ranking, columns = {f'{versus_string}'})
                
            else:
                next_match_df = Week1_Matches_table_df[Week1_Matches_table_df['Match_No'] == changing_Total_Matches + 1]
                next_versus_string = str(next_match_df['Team_1'].iloc[0]) + " vs " + str(next_match_df['Team_2'].iloc[0])

                finding_root_path_Exp_Team_df[f'{versus_string}'] = finding_root_path_Exp_Team_df[f'{next_versus_string}'].\
                    apply(lambda x: temp_df.iloc[x]['Linked_Prev_Match_Index']) 

            finding_root_path_Exp_Team_df[f"{versus_string} - Result"] = finding_root_path_Exp_Team_df[f'{versus_string}'].\
                apply(lambda curr_match_combo : define_curr_combo_result(temp_df.iloc[curr_match_combo]))

            changing_Total_Matches -= 1

        
        for versus_string in versus_strings[::-1]:
            Matchwise_Expected_Result_dict[f'{versus_string}'] = finding_root_path_Exp_Team_df[f'{versus_string} - Result'].unique().tolist()

        for versus_string in versus_strings[::-1]:
            if len(Matchwise_Expected_Result_dict[versus_string]) == 5:
                Matchwise_Expected_Result_dict[versus_string] = 'Does not matter. Just enjoy the game!'
            else:
                Matchwise_Expected_Result_dict[versus_string] = " (or) ".join(Matchwise_Expected_Result_dict[versus_string])
                
        Matchwise_Expected_Result_df = pd.Series(Matchwise_Expected_Result_dict).reset_index()
        Matchwise_Expected_Result_df.rename(columns = {'index': "Match", 0: "Favorable result"}, inplace=True)
        Matchwise_Expected_Result_df.index = np.arange(1, len(Matchwise_Expected_Result_df) + 1)
        Matchwise_Expected_Result_df = left_align(Matchwise_Expected_Result_df)

    return finding_root_path_Exp_Team_df, Matchwise_Expected_Result_df

## StoryBoard

> Let's imagine a Gujarat Giants Fan. The team is currently at 11th position (penultimate). She is wondering whether the team can move to 6th Position in the next six matches.. which increases the chances/hope to move to Playoffs

In [14]:
compute_chances_for_all_teams()

,JPP,PUP,UPY,BB,TTH,DD,HS,BW,UM,PAP,GG,TTI
1,36%,36%,-,-,-,-,-,-,-,-,-,-
2,64%,64%,-,-,-,-,-,-,-,-,-,-
3,-,-,60%,40%,-,-,-,-,-,-,-,-
4,-,-,40%,60%,-,-,-,-,-,-,-,-
5,-,-,-,-,100%,-,-,-,-,-,-,-
6,-,-,-,-,-,53%,0%,-,25%,-,0%,-
7,-,-,-,-,-,28%,20%,-,28%,1%,9%,-
8,-,-,-,-,-,16%,35%,-,21%,16%,14%,-
9,-,-,-,-,-,1%,28%,4%,12%,30%,22%,-
10,-,-,-,-,-,-,14%,34%,9%,19%,21%,-


> The compute table tells us that there is less than 1% chance for that to happen. But, nevertheless, possible

In [15]:
find_range_points('GG')

,Parameter,Value
0,Team_Name,Gujarat Giants
1,Top_Position,6.0
2,Worst_Position,11.0
3,Avg_Position,9.456
4,Mid_Position,10.0


> The range of points also tells us the same story.. So you wonder, which matches are important? Which teams should win for Gujarat Giants to move to 6th position

In [16]:
finding_root_path_Exp_Team_df, Matchwise_Expected_Result_df = Define_Favorable_Scenario(Exp_Team = 'GG', Exp_Ranking = 6)

display(Matchwise_Expected_Result_df)

,Match,Favorable result
1,UM vs GG,Easy win for GG against UM
2,BB vs UPY,Does not matter. Just enjoy the game!
3,PUP vs PAP,Does not matter. Just enjoy the game!
4,JPP vs HS,Easy win for JPP against HS (or) Close match with JPP winning against HS (or) Draw match between JPP and HS
5,UM vs DD,Easy win for UM against DD
6,GG vs TTI,Easy win for GG against TTI (or) Close match with GG winning against TTI


> This insightful table clearly tells the match wise expected favorable results needed for the team to be a strong contender in the race to reach the Playoffs

## The End!